In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

In [31]:
1132970-1129871

3099

This notebook calculates the PMI (in days) for each image. img_PMI = img_date - date_placed_ARF
* date_placed_ARF - the date a donor was placed at the ARF to start decomposing. Will be used as the time of death when calculating the PMI of an image. Can be found in the demography data. 
* img_date - the date an image was taken. Since it was discovered that this date was incorrectly mapped during old-to-new ID mapping, this date will be extracted from the old image file (e.g., UT01-18D_01_21_2018 (1).JPG). 

**Note**: Column names prefixed with 'old' represent data before old-to-new ID mapping, and names prefixed with 'new' after.

**Metadata**:
- Total # of donors: 841
    - \# of images: 1,376,741
- Total # of donors with a valid PMI: 691
    - \# of images: 1,132,970
    - Valid PMI means that the date placed at the ARF exists and is not negative
    - Saved as ../data/img_PMIs_no_negs.csv

**Notebook outputs**:
- mapping_metad_df3 (../data/img_PMIs.csv): images with a PMI, including outliers/noise (e.g., negative pmi).
- df_pmi2 (../data/img_PMIs_no_negs.csv): images with non-negative PMIs. Note, if a donor had at least one image with a negative PMI, all of its images were excluded, meaning it will be excluded from any downstream tasks.  

# Import old-to-new mapping metadata

In [2]:
mapping_metad_df = pd.read_csv('/da1_data/icputrd/arf/mean.js/public/anau_scripts/out/mapping_metadata.txt',
                              header=None, names=['new_id', 'new_img', 'old_path'])
display(mapping_metad_df.head())
print(mapping_metad_df.shape)
print(mapping_metad_df.info())

,new_id,new_img,old_path
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG


(1376741, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1376741 entries, 0 to 1376740
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   new_id    1376741 non-null  object
 1   new_img   1376741 non-null  object
 2   old_path  1376741 non-null  object
dtypes: object(3)
memory usage: 31.5+ MB
None


In [3]:
# create new_path column
mapping_metad_df['new_path'] = '/da1_data/icputrd/arf/mean.js/public/anau_img3/' + mapping_metad_df.new_id.str.cat(mapping_metad_df.new_img, sep='/')
mapping_metad_df.head()

,new_id,new_img,old_path,new_path
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG


In [4]:
# process old_path column into several columns which will be needed to derive the correct img date
mapping_metad_df['old_id'] = mapping_metad_df['old_path'].str.split('/', expand=True)[7]
mapping_metad_df['old_img'] = mapping_metad_df['old_path'].str.split('/', expand=True)[9]
mapping_metad_df['old_month'] = mapping_metad_df['old_img'].str.split('_', expand=True)[1]
mapping_metad_df['old_day'] = mapping_metad_df['old_img'].str.split('_', expand=True)[2]
mapping_metad_df['old_yr'] = mapping_metad_df['old_img'].str.split('_', expand=True)[3].str[:4]

display(mapping_metad_df.head())
print(mapping_metad_df.shape)
print(mapping_metad_df.info())

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,01,21,2018
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,01,21,2018
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,01,21,2018
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,01,21,2018
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,01,21,2018


(1376741, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1376741 entries, 0 to 1376740
Data columns (total 9 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   new_id     1376741 non-null  object
 1   new_img    1376741 non-null  object
 2   old_path   1376741 non-null  object
 3   new_path   1376741 non-null  object
 4   old_id     1376741 non-null  object
 5   old_img    1376741 non-null  object
 6   old_month  1376741 non-null  object
 7   old_day    1376741 non-null  object
 8   old_yr     1376741 non-null  object
dtypes: object(9)
memory usage: 94.5+ MB
None


In [5]:
# number of donor IDs
print(mapping_metad_df.old_id.unique().shape)

(841,)


In [6]:
# check unique values of old_month
print(mapping_metad_df.old_month.sort_values().unique())
print(len(mapping_metad_df.old_month.sort_values().unique()))

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12']
12


In [7]:
# check unique values of old_day
print(mapping_metad_df.old_day.sort_values().unique())
print(len(mapping_metad_df.old_day.sort_values().unique()))

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31']
31


In [8]:
# check unique values of old_yr
print(mapping_metad_df.old_yr.sort_values().unique())
print(len(mapping_metad_df.old_yr.sort_values().unique()))

['2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020'
 '2021' '2022']
12


In [9]:
# To fix the unwanted values, the following commands were executed in 
# /da1_data/icputrd/arf/mean.js/public/anau_scripts/out:

#sed -ibackup 's/_22\ (/_2022 (/g' mapping_metadata.txt
#sed -ibackup 's/_20\ (/_2020 (/g' mapping_metadata.txt
#sed -ibackup 's/__/_/g' mapping_metadata.txt
#sed -ibackup 's/\ 08_/08_(/g' mapping_metadata.txt
#sed -ibackup 's/_(16_/_16_/g' mapping_metadata.txt
#sed -ibackup 's/_5_/_05_/g' mapping_metadata.txt

In [10]:
# construct old_date (i.e., correct image date)
mapping_metad_df['old_date'] = mapping_metad_df.old_yr.str.cat(mapping_metad_df.old_month, sep='-').str.cat(mapping_metad_df.old_day, sep='-')
display(mapping_metad_df.head())
mapping_metad_df.info()

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,01,21,2018,2018-01-21
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,01,21,2018,2018-01-21
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,01,21,2018,2018-01-21
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,01,21,2018,2018-01-21
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,01,21,2018,2018-01-21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1376741 entries, 0 to 1376740
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   new_id     1376741 non-null  object
 1   new_img    1376741 non-null  object
 2   old_path   1376741 non-null  object
 3   new_path   1376741 non-null  object
 4   old_id     1376741 non-null  object
 5   old_img    1376741 non-null  object
 6   old_month  1376741 non-null  object
 7   old_day    1376741 non-null  object
 8   old_yr     1376741 non-null  object
 9   old_date   1376741 non-null  object
dtypes: object(10)
memory usage: 105.0+ MB


# Import date_place_ARF (PMI) data

In [11]:
date_placed_df = pd.read_pickle('/da1_data/icputrd/arf/mean.js/public/anau_demography/demography_processed_new.pkl')[['UTID', 'date_placed_ARF']]
display(date_placed_df.head())
print(date_placed_df.shape)
print(date_placed_df.info())

,UTID,date_placed_ARF
0,UT81-11D,NaT
1,UT80-11D,2012-01-09
2,UT54-11D,NaT
3,UT82-11D,2011-09-23
4,UT29-11D,NaT


(1069, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069 entries, 0 to 1068
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   UTID             1069 non-null   object        
 1   date_placed_ARF  876 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 16.8+ KB
None


# Join mapping_metad_df with date_placed_df

In [12]:
mapping_metad_df2 = pd.merge(mapping_metad_df, date_placed_df, left_on="old_id", right_on="UTID", how='left')
mapping_metad_df2.drop(['UTID'], axis=1, inplace=True)
display(mapping_metad_df2.head())
print(mapping_metad_df2.shape)
print(mapping_metad_df2.info())

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,01,21,2018,2018-01-21,2018-01-21
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,01,21,2018,2018-01-21,2018-01-21
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,01,21,2018,2018-01-21,2018-01-21
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,01,21,2018,2018-01-21,2018-01-21
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,01,21,2018,2018-01-21,2018-01-21


(1376741, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1376741 entries, 0 to 1376740
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   new_id           1376741 non-null  object        
 1   new_img          1376741 non-null  object        
 2   old_path         1376741 non-null  object        
 3   new_path         1376741 non-null  object        
 4   old_id           1376741 non-null  object        
 5   old_img          1376741 non-null  object        
 6   old_month        1376741 non-null  object        
 7   old_day          1376741 non-null  object        
 8   old_yr           1376741 non-null  object        
 9   old_date         1376741 non-null  object        
 10  date_placed_ARF  1156875 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(10)
memory usage: 126.0+ MB
None


In [13]:
# drop rows with missing value in date_placed_ARF column
mapping_metad_df3 = mapping_metad_df2[mapping_metad_df2.date_placed_ARF.notnull()].copy()
display(mapping_metad_df3.head())
print(mapping_metad_df2.shape[0] - mapping_metad_df3.shape[0])
print(mapping_metad_df3.shape)

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,01,21,2018,2018-01-21,2018-01-21
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,01,21,2018,2018-01-21,2018-01-21
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,01,21,2018,2018-01-21,2018-01-21
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,01,21,2018,2018-01-21,2018-01-21
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,01,21,2018,2018-01-21,2018-01-21


219866
(1156875, 11)


In [14]:
# number of donors without a date placed at ARF, meaning no PMI
mapping_metad_df2[mapping_metad_df2.date_placed_ARF.isnull()][['old_id', 'new_id']].nunique()

old_id    126
new_id    126
dtype: int64

In [15]:
# number of unique donors
print(mapping_metad_df3.new_id.unique().shape)

(715,)


In [16]:
display(mapping_metad_df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156875 entries, 0 to 1376740
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   new_id           1156875 non-null  object        
 1   new_img          1156875 non-null  object        
 2   old_path         1156875 non-null  object        
 3   new_path         1156875 non-null  object        
 4   old_id           1156875 non-null  object        
 5   old_img          1156875 non-null  object        
 6   old_month        1156875 non-null  object        
 7   old_day          1156875 non-null  object        
 8   old_yr           1156875 non-null  object        
 9   old_date         1156875 non-null  object        
 10  date_placed_ARF  1156875 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(10)
memory usage: 105.9+ MB


None

In [17]:
# convert dtypes
mapping_metad_df3['old_date'] = pd.to_datetime(mapping_metad_df3['old_date'], format='%Y-%m-%d', errors='coerce')
mapping_metad_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156875 entries, 0 to 1376740
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   new_id           1156875 non-null  object        
 1   new_img          1156875 non-null  object        
 2   old_path         1156875 non-null  object        
 3   new_path         1156875 non-null  object        
 4   old_id           1156875 non-null  object        
 5   old_img          1156875 non-null  object        
 6   old_month        1156875 non-null  object        
 7   old_day          1156875 non-null  object        
 8   old_yr           1156875 non-null  object        
 9   old_date         1156875 non-null  datetime64[ns]
 10  date_placed_ARF  1156875 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 105.9+ MB


In [18]:
# calculate pmi per image 
mapping_metad_df3['img_pmi_days'] = (mapping_metad_df3['old_date'] - mapping_metad_df3['date_placed_ARF']) / np.timedelta64(1,'D')
display(mapping_metad_df3.head())
display(mapping_metad_df3.info())

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF,img_pmi_days
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,01,21,2018,2018-01-21,2018-01-21,0.0
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,01,21,2018,2018-01-21,2018-01-21,0.0
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,01,21,2018,2018-01-21,2018-01-21,0.0
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,01,21,2018,2018-01-21,2018-01-21,0.0
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,01,21,2018,2018-01-21,2018-01-21,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156875 entries, 0 to 1376740
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   new_id           1156875 non-null  object        
 1   new_img          1156875 non-null  object        
 2   old_path         1156875 non-null  object        
 3   new_path         1156875 non-null  object        
 4   old_id           1156875 non-null  object        
 5   old_img          1156875 non-null  object        
 6   old_month        1156875 non-null  object        
 7   old_day          1156875 non-null  object        
 8   old_yr           1156875 non-null  object        
 9   old_date         1156875 non-null  datetime64[ns]
 10  date_placed_ARF  1156875 non-null  datetime64[ns]
 11  img_pmi_days     1156875 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 114.7+ MB


None

In [19]:
mapping_metad_df3.sample(5)

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF,img_pmi_days
939366,a95,a9510423.12.JPG,/da1_data/icputrd/arf/mean.js/public/2014/UT103-14D/Daily Photos/UT103-14D_04_23_2015 (12).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/a95/a9510423.12.JPG,UT103-14D,UT103-14D_04_23_2015 (12).JPG,04,23,2015,2015-04-23,2015-01-06,107.0
502235,397,39701202.39.JPG,/da1_data/icputrd/arf/mean.js/public/2013/UT71-13D/Daily Photos/UT71-13D_12_02_2013 (39).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/397/39701202.39.JPG,UT71-13D,UT71-13D_12_02_2013 (39).JPG,12,02,2013,2013-12-02,2013-10-29,34.0
152471,087,08700405.11.JPG,/da1_data/icputrd/arf/mean.js/public/2019/UT13-19D/Daily Photos/UT13-19D_04_05_2019 (11).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/087/08700405.11.JPG,UT13-19D,UT13-19D_04_05_2019 (11).JPG,04,05,2019,2019-04-05,2019-04-03,2.0
872535,9d7,9d710209.30.JPG,/da1_data/icputrd/arf/mean.js/public/2013/UT78-13D/Daily Photos/UT78-13D_02_09_2014 (30).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/9d7/9d710209.30.JPG,UT78-13D,UT78-13D_02_09_2014 (30).JPG,02,09,2014,2014-02-09,2013-12-19,52.0
1341073,f72,f7201111.21.JPG,/da1_data/icputrd/arf/mean.js/public/2014/UT72-14D/Daily Photos/UT72-14D_11_11_2014 (21).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/f72/f7201111.21.JPG,UT72-14D,UT72-14D_11_11_2014 (21).JPG,11,11,2014,2014-11-11,2014-08-01,102.0


In [20]:
mapping_metad_df3.describe()

,img_pmi_days
count,1.156875e+06
mean,5.388928e+01
std,7.836747e+01
min,-6.370000e+02
25%,1.600000e+01
50%,3.600000e+01
75%,6.700000e+01
max,2.359000e+03


In [21]:
# write to csv
mapping_metad_df3.to_csv('../data/img_PMIs.csv', index=False)
del mapping_metad_df, mapping_metad_df2, mapping_metad_df3

# Negative PMI analysis and removal
Negative PMIs cannot occurr. After consulting with the rest of the team, we decided to exclude these donors from the PMI estimation study.

In [22]:
df_pmi = pd.read_csv('../data/img_PMIs.csv')
print(df_pmi.shape)
display(df_pmi.head())

(1156875, 12)


,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF,img_pmi_days
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,1,21,2018,2018-01-21,2018-01-21,0.0


In [23]:
# number of donor IDs
print(df_pmi.new_id.unique().shape)

(715,)


In [24]:
# PMI cannot be negative, so analyze 
print(df_pmi[df_pmi.img_pmi_days < 0].shape)
print(df_pmi[df_pmi.img_pmi_days < 0].new_id.unique().shape)
print(df_pmi[df_pmi.img_pmi_days < 0].new_id.unique())
print(df_pmi[df_pmi.img_pmi_days < 0].old_id.unique())
print(df_pmi[df_pmi.img_pmi_days < 0].img_pmi_days.unique())

(890, 12)
(24,)
['009' '022' '02f' '034' '038' '053' '054' '073' '074' '09f' '0ab' '0e0'
 '0e4' '109' '10a' '11a' '121' '13f' '16b' '20f' '4c2' '844' '8ea' 'd83']
['UT101-18D' 'UT24-18D' 'UT34-18D' 'UT40-18D' 'UT43-18D' 'UT70-18D'
 'UT71-18D' 'UT95-18D' 'UT96-18D' 'UT36-19D' 'UT48-19D' 'UT98-19D'
 'UT92-16D' 'UT60-20D' 'UT61-20D' 'UT79-20D' 'UT101-21D' 'UT31-21D'
 'UT97-17D' 'UT86-16D' 'UT33-17D' 'UT51-17D' 'UT49-17D' 'UT78-17D']
[ -88.   -6.   -5.   -4.   -2.   -1. -637.  -24.  -42.  -34. -149. -326.
 -176. -288.  -76. -218. -109.  -12.  -11.  -10.   -9.   -8.   -7.   -3.
  -26.]


In [27]:
# drop imgs of donors that have at least one negative PMI
drop_ls = ['009','022', '02f', '034', '038', '053', '054', '073', '074', '09f', '0ab', '0e0', \
 '0e4', '109', '10a', '11a', '121', '13f', '16b', '20f', '4c2', '844', '8ea', 'c79', \
 'd83']
df_pmi2 = df_pmi[~df_pmi['new_id'].isin(drop_ls)].copy()
display(df_pmi2.head())
print('No. imgs dropped:', df_pmi.shape[0] - df_pmi2.shape[0])
display(df_pmi2.describe())

,new_id,new_img,old_path,new_path,old_id,old_img,old_month,old_day,old_yr,old_date,date_placed_ARF,img_pmi_days
0,000,00000121.01.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (1).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,UT01-18D,UT01-18D_01_21_2018 (1).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
1,000,00000121.02.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (2).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,UT01-18D,UT01-18D_01_21_2018 (2).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
2,000,00000121.03.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (3).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,UT01-18D,UT01-18D_01_21_2018 (3).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
3,000,00000121.04.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (4).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,UT01-18D,UT01-18D_01_21_2018 (4).JPG,1,21,2018,2018-01-21,2018-01-21,0.0
4,000,00000121.05.JPG,/da1_data/icputrd/arf/mean.js/public/2018/UT01-18D/Daily Photos/UT01-18D_01_21_2018 (5).JPG,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,UT01-18D,UT01-18D_01_21_2018 (5).JPG,1,21,2018,2018-01-21,2018-01-21,0.0


No. imgs dropped: 23905


,old_month,old_day,old_yr,img_pmi_days
count,1.132970e+06,1.132970e+06,1.132970e+06,1.132970e+06
mean,6.063891e+00,1.560305e+01,2.015503e+03,5.395654e+01
std,3.282373e+00,8.787254e+00,2.238731e+00,7.805390e+01
min,1.000000e+00,1.000000e+00,2.011000e+03,0.000000e+00
25%,3.000000e+00,8.000000e+00,2.014000e+03,1.600000e+01
50%,5.000000e+00,1.500000e+01,2.015000e+03,3.600000e+01
75%,9.000000e+00,2.300000e+01,2.017000e+03,6.700000e+01
max,1.200000e+01,3.100000e+01,2.022000e+03,2.359000e+03


In [28]:
# number of images 
print(df_pmi2.shape)

(1132970, 12)


In [29]:
# number of donor IDs
print(df_pmi2.new_id.unique().shape)

(691,)


In [30]:
# save as csv
df_pmi2.to_csv('../data/img_PMIs_no_negs.csv', index=False)